<a href="https://colab.research.google.com/github/tsj763/Alexnet_Artificial2/blob/main/%E1%84%8B%E1%85%B5%E1%86%AB%E1%84%80%E1%85%A9%E1%86%BC%E1%84%8C%E1%85%B5%E1%84%82%E1%85%B3%E1%86%BC2%E1%84%80%E1%85%AA%E1%84%8C%E1%85%A6_alexnet_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim # 최적화 알고리즘
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader

In [ ]:
epochs = 10 # 훈련 반복수
batch_size = 512 # 배치 크기

device = ("cuda" if torch.cuda.is_available() else "cpu") # device 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(torch.__version__)
print(device)

2.0.1+cu118
cuda


In [ ]:
transform = transforms.Compose([
    transforms.Resize(227),
    transforms.ToTensor()])

training_data = datasets.FashionMNIST(
    root="data",
    train=True, # train dataset
    download=True,
    transform=transform
)

validation_data = datasets.FashionMNIST(
    root="data",
    train=False, # test dataset
    download=True,
    transform=transform
)

100%|██████████| 26421880/26421880 [00:02<00:00, 12773391.68it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 198546.42it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3685100.23it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20622996.17it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
training_loader = DataLoader(training_data, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=64, shuffle=True)

In [ ]:
class fashion_mnist_alexnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(

            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0),

            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)

        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 1, 2),

            nn.ReLU(),
            nn.MaxPool2d(3, 2)

        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU()

        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU()

        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)

        )

        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x): # input size = 3x227x227
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out) # 64x4096x1x1
        out = out.view(out.size(0), -1) # 64x4096

        out = F.relu(self.fc1(out))
        out = F.dropout(out, 0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, 0.5)
        out = self.fc3(out)
        out = F.log_softmax(out, dim=1)

        return out

In [ ]:
model = fashion_mnist_alexnet().to(device)
criterion = F.nll_loss
optimizer = optim.Adam(model.parameters())

In [ ]:
# 훈련을 위한 함수 정리

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % 30 == 0:
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
# 테스트를 위한 함수 정리

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)  # -> mean
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        print('='*50)

In [ ]:
# 학습 시작
for epoch in range(1, epochs+1):
    train(model, device, training_loader, optimizer, epoch)
    test(model, device, validation_loader)

Train Epoch:1 [1856/60000 (3%)]	Loss: 0.221010
Train Epoch:1 [3776/60000 (6%)]	Loss: 0.437956
Train Epoch:1 [5696/60000 (9%)]	Loss: 0.110829
Train Epoch:1 [7616/60000 (13%)]	Loss: 0.381768
Train Epoch:1 [9536/60000 (16%)]	Loss: 0.150713
Train Epoch:1 [11456/60000 (19%)]	Loss: 0.261502
Train Epoch:1 [13376/60000 (22%)]	Loss: 0.184144
Train Epoch:1 [15296/60000 (25%)]	Loss: 0.262708
Train Epoch:1 [17216/60000 (29%)]	Loss: 0.240048
Train Epoch:1 [19136/60000 (32%)]	Loss: 0.241374
Train Epoch:1 [21056/60000 (35%)]	Loss: 0.276215
Train Epoch:1 [22976/60000 (38%)]	Loss: 0.252110
Train Epoch:1 [24896/60000 (41%)]	Loss: 0.367195
Train Epoch:1 [26816/60000 (45%)]	Loss: 0.571323
Train Epoch:1 [28736/60000 (48%)]	Loss: 0.247606
Train Epoch:1 [30656/60000 (51%)]	Loss: 0.314878
Train Epoch:1 [32576/60000 (54%)]	Loss: 0.233743
Train Epoch:1 [34496/60000 (57%)]	Loss: 0.351845
Train Epoch:1 [36416/60000 (61%)]	Loss: 0.270182
Train Epoch:1 [38336/60000 (64%)]	Loss: 0.242264
Train Epoch:1 [40256/60000 (